# Exploratory data analysis (EDA)
## Exploring Rossmann Pharmaceuticals' dataset 

## Importing the relevant libraries

In [1]:
#we will need the following libraries and modules
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
sns.set()